## Integralrechnung mit Python

In diesem Notebook lernen Sie die Grundlagen der Integralrechnung und einige Methoden in Python kennen, um Integrale (numerisch) berechnen zu können.

<b> Python Grundlagen: </b> Plots, NumPy, SymPy, SciPy<br>
<b> Math. Grundlagen: </b>  Analysis, Folgen, Grenzwerte, Stetigkeit, Riemann-Integral <br>

In [1]:
# Zum Berechnen von Funktionswerten und zum Plotten von Funktionsgraphen werden folgende Module benötigt
import numpy as np
import matplotlib.pyplot as plt
import micropip
await micropip.install("ipywidgets")
from IPython.display import display, Latex
from ipywidgets import interactive, widgets
from sympy import integrate, Symbol, sqrt, sin, cos, tan, E, pi, ln

# Hilfsmodul zur korrekten Darstellung in Latex von SymPy Ausgaben
from sympy.printing.latex import *

### <b> Inhaltsverzeichnis:</b> <br>
<ul>
    <li><a href="#flaecheninhalt">Der Flächeninhalt zwischen Funktionsgraph und x-Achse</a></li>
    <li><a href="#techniken">Integrationstechniken</a></li>
</ul>

### Der Flächeninhalt zwischen Funktionsgraph und $x$-Achse
<a id="flaecheninhalt"></a>


Was ist ein Integral? Wofür braucht man die Integralrechnung und <i>wie</i> berechnet man grundlegend Integrale? Darum soll es in diesem Abschnitt gehen. 

Man möchte den Inhalt der Fläche $A$ bestimmen, die zwischen dem Graphen der Funktion $f$ mit $f(x) = x^2$ und der $x$-Achse über dem Intervall $[0,1]$ eingeschlossen wird. Diese kann man über Rechtecke annähern, wie in folgender Code-Zelle nach Ausführung dargestellt wird. Dazu wird das Intervall $[0,1]$ in $n$ äquidistante Teile zerlegt, d.h. alle Teile sind gleich breit. Dann werden Rechtecke unterhalb des Graphen gebildet, die mit der linken oberen Ecke auf dem Graphen von $f$ liegen, da für die Höhe das Minimum des Funktionswertes auf jedem Teilintervall gewählt wird. Es findet daher eine Annäherung von unten statt. Was stellen Sie für den Flächeninhalt zwischen Graph und $x$-Achse fest, wenn Sie den Wert für $n$ anhand des Reglers verändern?

In [ ]:
# Sie müssen den Code dieser Zelle nicht nachvollziehen können!
def flaeche(n):
    plt.figure(figsize=(5, 5))

    # Intervall äquidistant zerlegen
    dx = (1 - 0) / n

    # Funktion definieren
    def f(x):
        return x**2

    # Untersumme der Rechtecke
    term = ""

    def summe(n):
        sum = 0
        for i in range(n):
            sum += f(0 + dx * n) * dx
            term += "f(x_i)"

        return sum

    # Graphen und Rechtecke plotten

    x = np.arange(0, 1, 0.01)
    ax = plt.subplot()
    plt.plot(x, f(x), color="b", linewidth=1)

    # Stützstellen festlegen
    xi = [0 + dx * (i + 1) for i in range(n)]

    for x in xi:
        rx = [x, x + dx, x + dx, x, x]
        ry = [0, 0, f(x), f(x), 0]
        plt.plot(rx, ry, color="r", linewidth=1)

    ax.set_aspect("equal")
    ax.axvline(x=0, color="k", ls="-")
    ax.axhline(y=0, color="k", ls="-")

    ax.text(
        0.05,
        0.9,
        r"$f(x) = x^2$",
        fontsize=14,
    )

    ax.text(
        0.05,
        0.7,
        rf"$n = {n}$",
        fontsize=14,
    )

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    ax.set_aspect("equal")

    plt.show()


interactive(
    flaeche,
    n=widgets.IntSlider(min=1, max=100, step=1, value=1),
)

Je größer $n$ wird, desto feiner wird die äquidistante Zerlegung des Intervalls $[0,1]$. Man spricht hier von einer sogenannten <i>Untersumme $U_n$</i>. Genauso gibt es eine Obersumme $O_n$, bei der die Rechteckhöhe dem Maximum von $f$ in jedem Teilintervall entspricht und so eine Annäherung von oben entsteht.

Streben in dem Fall die Ober- und Untersumme gegen denselben Grenzwert, dann nennt man $f$ <i>Riemann-integrierbar</i> und diesen Wert das <i>(Riemann-)Integral von $f$</i> auf dem Intervall $[0,1]$ und man schreibt $$ \lim_{n \to \infty } U_n = \lim_{n \to \infty} O_n = \int_0^1 f(x) \; \text{d}x $$

<div style= "color: black;background-color: powderblue ;margin: 10 px auto; padding: 10px; border-radius: 10px">
    <p style="font-size:12pt; text-align:center; color:   black; background-color: lightskyblue ;margin: 10 px auto; padding: 10px; border-radius: 10px" id="1"><b>Aufgabe 1</b>  </p> 

<b>Teilaufgabe 1.</b><br>
Berechnen Sie die Untersumme $U_n$ für $n = 1, 5$ und $10$ zur Funktion $f$ mit $f(x) = x^2$ auf dem Intervall $[0,1]$.

<hr>

<b>Teilaufgabe 2.</b><br>
Bestimmen Sie in Abhängigkeit von $n > 1$ eine Formel für die Untersumme $U_n$ aus Teilaufgabe 1. Überprüfen Sie Ihr Ergebnis in der STACK-Aufgabe "Untersumme einer quadratischen Funktion bestimmen".
<hr>

<b> Teilaufgabe 3.</b><br>
Zeigen Sie von Hand, dass die Untersumme $U_n$ konvergiert. Nehmen Sie an, dass die Obersumme gegen denselben Wert konvergiert und bestimmen Sie anschließend $\displaystyle \int_0^1 f(x) \; \text{d}x$. Wie lautet der Flächeninhalt zwischen dem Graphen von $f$ und der $x$-Achse auf dem Intervall $[0,1]$ in Flächeneinheiten (FE) ? 

## Integrationstechniken
<a id="techniken"></a>

Es ist mühselig, das Integral einer integrierbaren Funktion immerzu über die Ober- oder Untersumme zu bestimmen. Stattdessen sagt der <i>Hauptsatz der Differenzial- und Integralrechnung</i> aus, dass man zum Berechnen <i>Stammfunktionen</i> nutzen kann. Eine differenzierbare Funktion $F$ heißt dann <i>Stammfunktion</i> zu $f$, falls $F^\prime = f$ auf dem gemeinsamen (offenen) Definitionsbereich von $f$ gilt. Man kann zeigen, dass bereits alle stetigen Funktionen $f$ integrierbar sind, weshalb Sie in diesem Notebook in der Regel eine Stammfunktion finden werden und als geschlossenen Ausdruck angeben können. Beachten Sie allerdings, dass dies nicht immer gelten muss! Sie werden dazu im nachfolgenden Abschnitte Beispiele finden.

Hat man für $f$ eine Stammfunktion $F$ gefunden, dann gilt nach dem Hauptsatz der Differenzial- und Integralrechnung $$ \int_a^b f(x) \;\text{d}x = \left[F(x) \right]_a^b = F(b) - F(a). $$

So rechnet man nach, dass $$ \int_0^1 x^2 +x\; \text{d}x = \left[\frac{1}{3}x^3 + \frac{1}{2}x^2\right]_0^1 = \frac{1}{3} + \frac{1}{2} - 0 = \frac{5}{6}$$ gelten muss, denn $F(x) = \frac{1}{3}x^3 + \frac{1}{2}x^2$ ist eine Stammfunktion zu $f(x) = x^2 +x$. 


Zusätzlich hat das (Riemann-)Integral Eigenschaften, die es zu beachten gilt:

1) $\displaystyle \int_a^b c\cdot f(x) \;\text{d}x = c \cdot \int_a^b f(x) \;\text{d}x $

2) $\displaystyle \int_a^b f(x) + g(x) \;\text{d}x = \int_a^b f(x) \;\text{d}x + \int_a^b g(x) \;\text{d}x $.

Üben Sie einfache Integrale mit Hilfe der vorangestellten Eigenschaften in der nachfolgenden Übung. Überlegen Sie sich durch die umgekehrte Sichtweise, wie man eine Stammfunktion findet.

<div style= "color: black;background-color: powderblue ;margin: 10 px auto; padding: 10px; border-radius: 10px">
    <p style="font-size:12pt; text-align:center; color:   black; background-color: lightskyblue ;margin: 10 px auto; padding: 10px; border-radius: 10px" id="1"><b>Aufgabe 2</b>  </p> 

<b>Teilaufgabe 1:</b><br>
Berechnen Sie die folgenden Integrale von Hand:

(a) $ \displaystyle \int_0^1 x^2 -1\; \text{d}x $
   
(b) $ \displaystyle \int_0^\pi \cos(x) + \sin(x)\; \text{d}x $
   
(c) $ \displaystyle \int_{-3}^{-1} \frac{1}{x^3}\; \text{d}x $
<hr>

<b>Teilaufgabe 2. Stammfunktion eines Polynoms.</b><br>
Implementieren Sie eine Funktion <code>stammfunktionPolynom()</code>, die ein beliebiges Polynom als Parameter entgegennimmt und eine Stammfunktion dazu bildet und zurückgibt. 

<b>Anforderungen:</b> Das Polynom wird durch eine Liste der zugehörigen Koeffizienten übergeben. Beispielsweise ist für $f$ mit $f(x) = x^2 = 1 \cdot x^2 + 0 \cdot x + 0$ der Parameter gegeben durch <code>[1, 0, 0]</code>. Ebenso soll eine Stammfunktion als Liste mit den zugehörigen Koeffizienten zurückgegeben werden. 

<b>Beispiel:</b> Es sollte <code>stammfunktionPolynom([1, 0, 0]) = [0.3333333333333333, 0, 0, 0]</code> ergeben.

Ergänzen Sie dazu den Code an den entsprechenden Stellen.

In [12]:
def stammfunktionPolynom(p):
    stammfunktion = p + [0]

    # Bis wohin muss i zählen?
    for i in range(0, ...):
        # Welche Bedingung sollte p[i] erfüllen?
        if p[i] ... 0:

            # Wie lautet der neue Wert für stammfunktion[i] ?
            stammfunktion[i] = ...

    return stammfunktion

# Testen Sie die Funkion
stammfunktionPolynom([1, 0, 0])

[0.3333333333333333, 0, 0, 0]

> **Am Rande:** Mithilfe des Integrals können Flächeninhalte zwischen der $x$-Achse und Graphen von (integrierbaren) Funktionen ermittelt werden. Das Integral selbst ist abstrakt gesehen ein weitaus stärkeres Werkzeug, womit man viele weitere Phänomena berechnen kann. Ein Beispiel dazu sehen Sie in dem nachfolgendem Abschnitt. Ein weiteres Anwendungsszenario ist die Rekonstruktion aus Daten. So wie das Ableiten einer Funktion des Weges die momentane Geschwindigkeit ergibt, kann umgekehrt das Integrieren einer Geschwindigkeitsfunktion den zurückgelegten Weg ermitteln.

### Integrieren mit Python

Mithilfe von Python kann man Stammfunktionen bestimmen lassen und ebenso endliche Integrale berechnen. Das Modul <code>SymPy</code> findet dafür Verwendung.

In [ ]:
# Festlegung der symbolischen Variable zum Integrieren
x = Symbol("x")

# Integrationsbefehl von SymPy
stammfunktion = integrate(x**2 + 1, x)

display(Latex(rf"$\displaystyle \int x^2 +1 \; \text{{d}}x = {latex(stammfunktion)}$"))

Um ein bestimmtes Integral zu berechnen, gibt man beim <code>integrate</code> Befehl die Grenzen mit an. Achten Sie auf die zusätzliche Klammerschreibweise.

In [ ]:
# Festlegung der symbolischen Variable zum Integrieren
x = Symbol("x")

# Integrationsbefehl von SymPy über die Funktion f(x) = x + 1 mit Grenzen a = 0 und b = 1
integral = integrate(x + 1, (x, 0, 1))

# Ausgabe in der Konsole
display(Latex(rf"$ \displaystyle \int_0^1 x + 1 \; \text{{d}}x = {latex(integral)}$"))

<div style= "color: black;background-color: powderblue ;margin: 10 px auto; padding: 10px; border-radius: 10px">
    <p style="font-size:12pt; text-align:center; color:   black; background-color: lightskyblue ;margin: 10 px auto; padding: 10px; border-radius: 10px" id="1"><b>Aufgabe 3</b>  </p> 

Überprüfen Sie Ihre Ergebnisse aus <b>Aufgabe 2 Teilaufgabe 1</b>, indem Sie mithilfe von <code>SymPy</code> jeweils eine Stammfunktion bilden und anschließend die Integrale berechnen lassen. Ergänzen Sie dazu die Code-Zelle an den entsprechenden Stellen.


In [ ]:
# Festlegung der symbolischen Variable zum Integrieren
x = Symbol("x")

# Funktion aus (a)
stammfunktionf1 = integrate(x**2 - 1, (x, ..., ...))

# Funktion aus (b)
stammfunktionf2 = integrate(cos(x) + sin(x), ...)

# Funktion aus (c)
stammfunktionf3 = integrate(..., ...)

# Ausgabe der Stammfunktionen aus (a), (b) und (c)
display(
    Latex(
        rf"$ \displaystyle \int_0^1 x^2 - 1 \; \text{{d}}x = {latex(stammfunktionf1)}$"
    )
)
display(
    Latex(
        rf"$ \displaystyle \int_0^\pi \cos(x) + \sin(x) \; \text{{d}}x = {latex(stammfunktionf2)}$"
    )
)
display(
    Latex(
        rf"$ \displaystyle \int_{{-3}}^{{-1}} \frac{{1}}{{x^3}} \; \text{{d}}x = {latex(stammfunktionf3)}$"
    )
)

Jedoch ist es nicht immer möglich, für stetige Funktionen einen geschlossenen Ausdruck einer Stammfunktion zu finden. Betrachten Sie dazu folgendes Beispiel:

Es sei $f : [a,b] \longrightarrow \mathbb{R} $ eine stetig-differenzierbare Funktion, d.h. $f$ ist differenzierbar auf $(a,b)$ und die Ableitung $f'$ ist selbst wieder stetig. Dann lässt sich mit der Formel $$ L(f) = \int_{a}^{b} \sqrt{1 + f'(x)^2} \;\text{d}x $$ die Länge des Graphen zwischen den Punkten $(a, f(a))$ und $(b, f(b))$ bestimmen. Möchte man beispielsweise die Länge der Sinuskurve vom Startpunkt $(0,0)$ bis hin zum Punkt $(0, \pi)$ berechnen, so muss man das Integral $$ L(f) = \int_{0}^{\pi} \sqrt{1 + \cos^2(x)} \;\text{d}x $$ lösen. Da die Cosinusfunktion stetig ist, existiert dieses Integral, der Integrand besitzt jedoch keine Stammfunktion, die durch elementare Funktionen ausgedrückt werden kann. In dem Fall muss man solche Integrale numerisch lösen. Betrachten Sie dazu nachfolgende Code-Zelle und beobachten Sie, was passiert, wenn <code>SymPy</code> nach Ausführung eine Stammfunktion bestimmen soll.

In [21]:
# Festlegung der symbolischen Variable
x = Symbol("x")

# Integrationsbefehl von SymPy
integrate(sqrt(1 + cos(x) ** 2), x)

Integral(sqrt(cos(x)**2 + 1), x)

Nach erfolgreicher Ausführung der Code-Zelle gibt das Modul <code>SymPy</code> lediglich das Integral selbst als Stammfunktion an, was korrekt bleibt, aber nicht die Frage der eigentlichen Berechnung löst. Daher muss dieses Integral offenbar numerisch gelöst werden, was in Python mit dem Modul <code>SciPy</code> gelingt.

In [22]:
# Import der benötigten Funktion aus SciPy
from scipy.integrate import quad

# Hier wird der Integrand mit der Integrationsvariablen festgelegt
f = lambda x: sqrt(1 + cos(x) ** 2)

# Integrationsbefehl von SciPy
quad(f, 0, pi)

(3.8201977890277115, 1.3015768476030976e-13)

Die erste Komponente gibt dabei das numerische Ergebnis des Integrals an, die zweite Komponente den geschätzen Fehlerwert zum exakten Ergebnis. In dem Beispiel ist also die Länge der Sinuskurve vom Ursprung bis zum Punkt $(\pi,0)$ etwa $$\int_0^\pi \sqrt{1+\cos^2(x)} \; \text{d} x \approx 3.820198$$ Längeneinheiten lang, wobei der geschätze Fehlerwert zur exakten Länge bei ca. $1{,}30158 \cdot 10^{-13}$ liegt.

<div style= "color: black;background-color: powderblue ;margin: 10 px auto; padding: 10px; border-radius: 10px">
    <p style="font-size:12pt; text-align:center; color:   black; background-color: lightskyblue ;margin: 10 px auto; padding: 10px; border-radius: 10px" id="1"><b>Aufgabe 4</b>  </p> 

Berechnen Sie die folgenden Integrale mithilfe von Python und überprüfen Sie Ihre Ergebnisse in der STACK-Aufgabe "Bestimmte Integrale berechnen (2)".

(a) $ \displaystyle \int_0^\pi \frac{\sin(x)}{x} \; \text{d} x$

(b) $ \displaystyle \int_1^2 e^{-x^2} \; \text{d} x$

(c) $ \displaystyle \int_e^{5e} \sin \left(\sqrt{1+ \ln^2(x)} \right) \; \text{d} x$

In [ ]:
from scipy.integrate import quad

# Integral aus (a)
f1 = lambda x: ...
result1 = quad(f1, ..., ...)

# Integral aus (b)
f2 = ...
result2 = quad(..., ..., ...)

# Integral aus (c)
f3 = ...
result3 = ...

# Ausgabe der numerischen Ergebnisse
display(
    Latex(
        rf"$ \displaystyle \int_0^\pi \frac{{\sin(x)}}{{x}} \; \text{{d}} x = {result1[0]}$"
    )
)
display(Latex(rf"$ \displaystyle \int_1^2 e^{{-x^2}} \; \text{{d}} x = {result2[0]}$"))

display(
    Latex(
        rf"$ \displaystyle \int_e^{{5e}} \sin \left(\sqrt{{1+ \ln^2(x) }} \right) \; \text{{d}} x = {result3[0]}$"
    )
)

### Uneigentliche Integrale

Von besonderem Interesse sind <i>uneigentliche Integrale</i>, bei denen Funktionen über Intervalle wie $[0, \infty)$ oder $(0,1]$ integriert werden müssen. Die Frage stellt sich dabei, ob solchen Integralen überhaupt ein Wert zugeschrieben werden kann. Führen Sie nachfolgende Code-Zelle aus und beobachten Sie beim verkleinern der unteren Grenze des Integrals, wie sich der Flächeninhalt verhält.

In [ ]:
def uneigentlichesIntegral(a):
    plt.figure(figsize=(5, 10))

    # Graphen plotten
    x = np.arange(0.01, 5, 0.01)

    def f(x):
        return 1 / x

    y = f(x)
    x_ = np.arange(a + 0.01, 5, 0.01)

    ax = plt.subplot()
    plt.plot(x, y, color="b", linewidth=1)
    ax.fill_between(x_, 0, f(x_), alpha=0.5)

    ax.set_aspect("equal")
    ax.axvline(x=0, color="k", ls="-")
    ax.axvline(x=a, color="k", ls="--", linewidth=1)

    ax.axhline(y=0, color="k", ls="-")

    plt.xlim(0, 5)
    plt.ylim(0, 5)
    ax.text(
        a + 0.2,
        4,
        rf"$\int_{{{a}}}^5 1/x \; \text{{d}}x$",
        fontsize=14,
    )
    # symbolische Integration
    x = Symbol("x")
    flaeche = integrate(1 / x, (x, a, 5))

    print(
        rf"Die Fläche unterhalb des Funktionsgraphen zu 1/x von a = {a} bis 5 ist gleich {flaeche}"
    )

    plt.show()
    ax.set_aspect("equal")


interactive(
    uneigentlichesIntegral,
    a=widgets.FloatSlider(min=0, max=1, step=0.01, value=1),
)

Für die Funktion $f$ mit $f(x) = \frac{1}{x}$ auf dem Intervall $(0,\infty)$ gibt es keinen endlichen Flächeninhalt auf dem Intervall $[a, 5]$, wenn sich die untere Grenze $a$ der $0$ annähert. Das kann man auch analytisch nachweisen, indem man solche uneigentlichen Integrale über einen Grenzwertprozess berechnet. Man definiert $$ \int_0^5 \frac{1}{x} \; \text{d}x = \lim_{a \to 0^+} \; \int_a^5 \frac{1}{x} \; \text{d}x = \lim_{a \to 0^+} \; [\ln(x)]_a^5 = \lim_{a \to 0^+} \ln(5) - \ln(a) = \infty.$$

Dagegen erhält man einen endlichen Wert, wenn wir den Integranden etwas abändern! Für $f$ mit $f(x) = \frac{1}{\sqrt{x}}$ gilt auf demselben Intervall $$ \int_0^5 \frac{1}{\sqrt{x}} \; \text{d}x = \lim_{a \to 0^+} \; \int_a^5 \frac{1}{\sqrt{x}} \; \text{d}x = \lim_{a \to 0^+} \; \left[2\sqrt{x}\right]_a^5 = \lim_{a \to 0^+} 2\sqrt{5} - 2\sqrt{a} = 2\sqrt{5} < \infty .$$

Probieren Sie den Regler für nachfolgende Code-Zelle nach Ausführung aus!

In [ ]:
def uneigentlichesIntegral(a):
    plt.figure(figsize=(5, 10))

    # Graphen plotten
    x = np.arange(0.01, 5, 0.01)

    def f(x):
        return 1 / (x ** (1 / 2))

    y = f(x)
    x_ = np.arange(a + 0.01, 5, 0.01)

    ax = plt.subplot()
    plt.plot(x, y, color="b", linewidth=1)
    ax.fill_between(x_, 0, f(x_), alpha=0.5)

    ax.set_aspect("equal")
    ax.axvline(x=0, color="k", ls="-")
    ax.axhline(y=0, color="k", ls="-")

    plt.xlim(0, 5)
    plt.ylim(0, 5)
    ax.text(
        a + 0.2,
        4,
        rf"$\int_{{{a}}}^5 \; \frac{{1}}{{\sqrt{{x}}}} \; \text{{d}}x$",
        fontsize=14,
    )

    # symbolische Integration
    x = Symbol("x")
    flaeche = integrate(1 / (x ** (1 / 2)), (x, a, 5))

    print(
        rf"Die Fläche unterhalb des Funktionsgraphen zu 1/sqrt(x) von a = {a} bis 5 ist gleich {flaeche}"
    )

    plt.show()
    ax.set_aspect("equal")


interactive(
    uneigentlichesIntegral,
    a=widgets.FloatSlider(min=0, max=1, step=0.01, value=1),
)

Bearbeiten Sie zum Abschluss die Aufgabe 5.

<div style= "color: black;background-color: powderblue ;margin: 10 px auto; padding: 10px; border-radius: 10px">
    <p style="font-size:12pt; text-align:center; color:   black; background-color: lightskyblue ;margin: 10 px auto; padding: 10px; border-radius: 10px" id="1"><b>Aufgabe 5</b>  </p> 

Bearbeiten Sie die STACK-Aufgaben "Uneigentliches Integral (1)" und "Uneigentliches Integral (2)".